In [3]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize

## Group Assignment
### Team Number: 11
### Team Member Names: Akram, Annie, Jester
### Team Strategy Chosen: Market Beat

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


## **General Strategy for the Project**:

#### Initialization
0) Define and initialize necessary global variables

#### Part #1: Data Filtering and Cleaning
1) Filter out all valid US and CAD Stocks from the provided CSV file.
2) Download and store closing price, options and volume data in a dictionary using yfinance, where US stocks prices are converted using real-time exchange rate.
3) Filter out tickers within date range based on given minimum monthly average volume and minimum trading days in a month.

#### Part #2: Portfolio Construction
4) Rank stocks based on Standard Deviation of percentage change in returns in descending order.
5) Rank stocks based on PCR values using options data in descending order.
6) Score the stocks based on the two ranks, and create a new ranking based on the scoring.
7) We select stocks based on the ranking and calculate weights that would maximize the portfolio sharpe ratio while beta is within pre-defined constraints.
8) Run sharpe ratio calculation function on portfolio from size 12(min) to 24(max), so that the final portfolio (with weightings) is picked based on highest output sharpe ratio.

#### Part #3: Evaluation and Proof
9) Given the chosen portfolio of xx stocks, graph the change in portfolio standard deviation as other stocks are added.
10) Beta of portfolio compared to the S&P 500.
11) Calculate the Beta between our portfolio and an equally weighted portfolio including all valid stocks.
12) Beta between our portfolio with varied weight versus when the portfolio is equally weighted.
13) Sharpe ratio between our portfolio with varied weight versus when it is equally weighted.
14) Graphically compare sharpe ratios amongst our portfolio of xx stocks and the portfolios of varying 12-24 stocks.

#### Part #4: Final Output
15) Creating the final portfolio dataframe and CSV.

## Initializing Variables

In [ ]:
def get_tickers():
    tickers = pd.read_csv('Tickers.csv')
    ticker_lst = [tickers.columns[0]] + (list(tickers[tickers.columns[0]]))
    return ticker_lst

In [5]:
# Important Constants: 
amount = 1_000_000 # Initial investment amount of $1,000,000
group = 11

# Define constants
min_avg_volume = 100000
min_trading_days = 18
start_date, end_date = '2022-09-30', '2024-09-30'
min_stocks, max_stocks = 12, 24

# Reading in CSV file: 
tickers = pd.read_csv('Tickers.csv')
ticker_lst = get_tickers()

# Initializing variable to store the tickers we will use in our portfolio
columns = ['Ticker', 'Price', 'Currency', 'Shares', 'Value', 'Weight']
Portfolio_Final = pd.DataFrame(columns=columns)
exchange_rate = yf.Ticker('CAD=X').fast_info['last_price']
print(f'The current exchange rate for the latest available day:\nUSD -> CAD: ${np.round(exchange_rate, 4)}')

The current exchange rate for the latest available day:
USD -> CAD: $1.3969


#### We must filter the tickers csv as follows:
- Must be listed on yfinance
- The currency is listed as USD or CAD 
- 100,000+ average monthly volume trades
- More than 18 trades per month
- Sufficient data

In [6]:
# Filtering valid stocks by inputting a list of strings for each ticker. 
def filter_stocks(ticker_lst):
    valid_tickers, invalid_tickers = {}, []
    # Loop through all tickers to check if they are valid
    for ticker in ticker_lst:
        stock = yf.Ticker(ticker)
        try:
            info = stock.fast_info # Get basic stock info

            hist = stock.history(start=start_date, end=end_date) # Get stock history
            hist.index = hist.index.strftime('%Y-%m-%d')

            avg_volume = hist.loc[((hist.index >= start_date) & (hist.index <= end_date))]['Volume'].mean() # Calculate average volume in specified date range.
            currency = info.get("currency")
            if ((hist.empty is not None) and # filter for stocks delisted on yfinance
                ( currency == "USD" or currency == "CAD") and # filter for stocks that are not USD
                (avg_volume >= min_avg_volume)): # Filter by volume greater than 100,000
                if currency == "CAD":
                    valid_tickers[ticker] = hist['Close'] # Store the close prices of the stock as a Series
                elif currency == "USD":
                    valid_tickers[ticker] = hist['Close'] * exchange_rate # Convert USD to CAD
            else:
                invalid_tickers.append(ticker)
        except:
            invalid_tickers.append(ticker)
    return [valid_tickers, invalid_tickers]
    # valid_tickers is a dictionary of Series where the key is the name of the ticker. 
    # invalid_tickers is a list of ticker strings which were removed in the filtering process. 

In [8]:
def calculate_std(data):
    data.index = pd.to_datetime(data.index)
    
    # Calculate daily percentage returns
    returns = data.pct_change().dropna()

    # Calculate standard deviation of returns
    std = pd.DataFrame(returns.std(), columns=['Standard Deviation'])

    # Sort by standard deviation
    std_sorted = std.sort_values(by='Standard Deviation', ascending=False)

    # Add Rank column
    std_sorted['Rank'] = range(len(std_sorted))

    # Add Score column
    highest_std_value = std_sorted['Standard Deviation'].iloc[0]
    std_sorted['Score'] = (std_sorted['Standard Deviation'] / highest_std_value) * 100

    return std_sorted

In [9]:
def calculate_return(data):
    data.index = pd.to_datetime(data.index)
    
    # Calculate daily percentage returns
    returns = data.pct_change().dropna()

    # Calculate standard deviation of returns
    ret = pd.DataFrame(returns.mean(), columns=['Return'])

    # Sort by standard deviation
    ret_sorted = ret.sort_values(by='Return', ascending=False)

    # Add Rank column
    ret_sorted['Rank'] = range(len(ret_sorted))

    # Add Score column
    highest_ret_value = ret_sorted['Return'].iloc[0]
    ret_sorted['Score'] = (ret_sorted['Return'] / highest_ret_value) * 100

    return ret_sorted

In [10]:
# Loading data into variables
stock_filter = filter_stocks(ticker_lst)
ticker_data = stock_filter[0]
ticker_lst = list(ticker_data.keys()) # Reassign original ticker list
data = pd.DataFrame()
for ticker in ticker_data:
    data[ticker] = ticker_data[ticker]

# returns = data.pct_change()
# returns.drop(index=returns.index[0], inplace = True)

data.head()

$AGN: possibly delisted; no timezone found
$CELG: possibly delisted; no timezone found
$MON: possibly delisted; no timezone found
$RTN: possibly delisted; no timezone found


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-30,190.722100,172.088638,129.307490,346.654665,63.391801,157.846312,182.810035,169.133024,39.885125,6.51,...,150.561608,113.588989,37.189999,24.323668,76.140358,202.210065,684.811306,259.594404,205.115578,50.945604
2022-10-03,196.587266,177.358648,132.995886,356.880573,65.661513,161.869294,189.734342,176.075470,41.060550,6.53,...,156.212019,114.575356,37.799999,25.033070,77.623253,208.820600,699.008199,266.843097,207.083699,52.525012
2022-10-04,201.624448,182.064431,137.753378,369.571914,69.039388,169.146985,197.105849,186.496108,42.764253,6.74,...,163.075100,117.543640,42.610001,25.316833,79.079193,215.757803,709.394734,273.665414,214.448258,53.864354
2022-10-05,202.038440,183.782575,138.154294,369.612284,68.491988,168.951424,195.628815,184.540499,42.156730,6.67,...,166.433345,117.041313,41.959999,25.104012,78.620865,219.219796,714.683086,267.322802,213.229258,53.321036
2022-10-06,200.699823,179.884592,136.911464,363.051028,67.771013,168.043467,192.932291,184.666212,41.549210,6.64,...,166.953079,113.762512,41.320000,24.421211,75.751678,217.573695,703.916768,264.111510,210.270757,51.905883


In [37]:
# Function to get the total volume for a call or put of a given stock.
# ticker: yfinance Ticker class
# put: Boolean for if you want to calculate put volume. Else, put False for call volume. 
def get_options_vol(ticker, put):
    exps = ticker.options # Expiration dates of available options
    optdata = pd.DataFrame() # Data storage
    for exp in exps:
        chain = pd.DataFrame()
        if put: chain = ticker.option_chain(exp).puts['volume'] # Gets the desired columns
        else: chain = ticker.option_chain(exp).calls['volume'] # If put options are desired then use this data.
        optdata = pd.concat([optdata, chain]) # Add the calls/puts to the main dataframe. 
    return optdata.sum()['volume'] # output total volue of put/call options

# Function to calculate the PCR for each stock. 
def PCR_calc(tickers):
    pcrdata = pd.DataFrame(columns=['Ticker', 'Put Volume', 'Call Volume', 'PCR'])
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        try: 
            # Get the volume for Put and Call options:
            call_options = get_options_vol(stock, False)
            put_options = get_options_vol(stock, True)
            # Calculate PCR Ratio:
            pcr = call_options / put_options # Order reversed from the formula for sake of ranking
            #print(f"Ticker: {ticker}, PCR: {pcr}")  # Debugging
            pcrdata.loc[len(pcrdata)] = [ticker, put_options, call_options, pcr]
        except Exception as e:
            print(f"Options Data Not Found {ticker}: {e} not found")  # Debugging (output error)
            pass
    return pcrdata

In [12]:
std = calculate_std(data)
std

C:\Users\jeste\AppData\Local\Temp\ipykernel_40772\2355861676.py:5: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change().dropna()


,Standard Deviation,Rank,Score
SHOP.TO,0.036119,0,100.000000
BB.TO,0.034555,1,95.669034
PYPL,0.023802,2,65.898661
QCOM,0.023472,3,64.984556
AMZN,0.021319,4,59.023566
USB,0.021262,5,58.864426
BA,0.020383,6,56.431218
LLY,0.018070,7,50.027405
CAT,0.017667,8,48.911589
TXN,0.016711,9,46.266552


In [21]:
ret = calculate_return(data)
ret

C:\Users\jeste\AppData\Local\Temp\ipykernel_40772\3450711558.py:5: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change().dropna()


,Return,Rank,Score
SHOP.TO,0.002754,0,100.000000
LLY,0.002195,1,79.701690
CAT,0.001972,2,71.625795
AXP,0.001584,3,57.531870
BK,0.001484,4,53.880918
BLK,0.001307,5,47.449886
AMZN,0.001244,6,45.182508
QCOM,0.001189,7,43.163813
AAPL,0.001145,8,41.591738
C,0.001095,9,39.772917


In [38]:
# Load the PCR values for each of the valid stocks into a variable
options_data = PCR_calc(ticker_lst)
options_data = options_data.sort_values(by='PCR', ascending=False)
options_data['Rank'] = [i for i in range(len(options_data))]
highest_pcr = options_data['PCR'].iloc[0]
options_data['Score'] = (options_data['PCR'] / highest_pcr) * 100
options_data.set_index('Ticker', inplace=True)


# Display the table of rankings based off PCR. 
# The rankings are based off the stocks with the greatest sentiment for if they will go up or not
# The tickers at the top of the list have a high call rate (meaning the price will go up)
pcr = options_data
pcr

Options Data Not Found BB.TO: 'volume' not found
Options Data Not Found RY.TO: 'volume' not found
Options Data Not Found SHOP.TO: 'volume' not found
Options Data Not Found T.TO: 'volume' not found
Options Data Not Found TD.TO: 'volume' not found


,Put Volume,Call Volume,PCR,Rank,Score
Ticker,,,,,
PEP,5763.0,35932.0,6.234947,0,100.000000
KO,13171.0,50030.0,3.798497,1,60.922677
PM,3092.0,8275.0,2.676261,2,42.923561
PYPL,12469.0,31317.0,2.511589,3,40.282439
PG,7077.0,16438.0,2.322736,4,37.253494
BA,27272.0,57991.0,2.126393,5,34.104433
AMZN,163766.0,339482.0,2.072970,6,33.247595
LMT,4332.0,8372.0,1.932595,7,30.996168
MRK,9362.0,18016.0,1.924375,8,30.864338


In [39]:
def calculate_scoreboard(std, pcr, ret):
    """
    Merges three DataFrames (std, pcr, ret) on their index (assumed to be ticker names),
    calculates the average of their 'Score' columns, and sorts the result by 'Average Score'.
    Handles NaN values by taking the value from 'ret' where available.
    """
    # Merge std and pcr DataFrames
    merged = std[['Score']].merge(
        pcr[['Score']], left_index=True, right_index=True, suffixes=('_std', '_pcr'), how='outer'
    )
    
    # Merge the resulting DataFrame with ret
    merged = merged.merge(
        ret[['Score']].rename(columns={'Score': 'Score_ret'}),  # Rename the Score column in ret
        left_index=True,
        right_index=True,
        how='outer'
    )

    # Fill missing values: take the value from 'ret' where other columns are NaN
    merged['Score_std'] = merged['Score_std'].fillna(merged['Score_ret'])
    merged['Score_pcr'] = merged['Score_pcr'].fillna(merged['Score_ret'])
    merged['Score_ret'] = merged['Score_ret'].fillna(merged[['Score_std', 'Score_pcr']].mean(axis=1))

    # Calculate the average score from all three columns
    merged['Average Score'] = merged[['Score_std', 'Score_pcr', 'Score_ret']].mean(axis=1)

    # Sort the DataFrame by 'Average Score' in descending order
    merged_sorted = merged.sort_values(by='Average Score', ascending=False)

    return merged_sorted



calculate_scoreboard(std, pcr, ret)

,Score_std,Score_pcr,Score_ret,Average Score
SHOP.TO,100.000000,100.000000,100.000000,100.000000
LLY,50.027405,26.993368,79.701690,52.240821
AMZN,59.023566,33.247595,45.182508,45.817890
PEP,28.227778,100.000000,9.004320,45.744033
CAT,48.911589,13.300584,71.625795,44.612656
QCOM,64.984556,23.446427,43.163813,43.864932
AXP,45.198283,17.848506,57.531870,40.192886
BA,56.431218,34.104433,26.129494,38.888382
BK,40.250415,22.459591,53.880918,38.863642
AAPL,43.656567,25.645929,41.591738,36.964745


In [40]:
def calculate_scoreboard_2(std, pcr, ret):
    """
    Merges three DataFrames (std, pcr, ret) on their index (assumed to be ticker names),
    calculates the average of 'Score_std' and 'Score_pcr', using 'Score_ret' only when 'Score_pcr' is NaN.
    The average score is strictly based on 'Score_std' and 'Score_pcr'.
    """
    # Merge std and pcr DataFrames
    merged = std[['Score']].merge(
        pcr[['Score']], left_index=True, right_index=True, suffixes=('_std', '_pcr'), how='outer'
    )
    
    # Merge the resulting DataFrame with ret
    merged = merged.merge(
        ret[['Score']].rename(columns={'Score': 'Score_ret'}),  # Rename the Score column in ret
        left_index=True,
        right_index=True,
        how='outer'
    )

    # Use 'Score_ret' where 'Score_pcr' is NaN
    merged['Score_pcr'] = merged['Score_pcr'].fillna(merged['Score_ret'])
    merged.drop(columns=['Score_ret'], inplace=True)

    # Calculate the average score using only 'Score_std' and 'Score_pcr'
    # Exclude rows where both 'Score_std' and 'Score_pcr' are NaN
    merged['Average Score'] = merged[['Score_std', 'Score_pcr']].mean(axis=1)

    # Sort the DataFrame by 'Average Score' in descending order
    merged_sorted = merged.sort_values(by='Average Score', ascending=False)

    
    return merged_sorted




calculate_scoreboard_2(std, pcr, ret)

,Score_std,Score_pcr,Average Score
SHOP.TO,100.000000,100.000000,100.000000
PEP,28.227778,100.000000,64.113889
PYPL,65.898661,40.282439,53.090550
AMZN,59.023566,33.247595,46.135581
BA,56.431218,34.104433,45.267825
QCOM,64.984556,23.446427,44.215491
KO,24.033257,60.922677,42.477967
LLY,50.027405,26.993368,38.510386
PM,29.544189,42.923561,36.233875
UPS,45.410217,26.784566,36.097392


In [16]:
# Define to get call and put option data (specifically the total volume)
# def get_options_vol(ticker, put):
#     exps = ticker.options # Expiration dates of available options
#     data = pd.DataFrame() # Data storage
#     for exp in exps:
#         chain = pd.DataFrame()
#         if put: chain = ticker.option_chain(exp).puts['volume'] # Gets the desired columns
#         else: chain = ticker.option_chain(exp).calls['volume'] # If put options are desired then use this data.
#         data = pd.concat([data, chain]) # Add the calls/puts to the main dataframe. 
#     return data.sum()['volume'] # output total volue of put/call options

#cols = ['lastTradeDate','strike', 'bid', 'ask', 'volume', 'inTheMoney', 'currency']
# chain = chain.set_index('lastTradeDate') # Reset the index to the expiration dates
# chain.index = chain.index.strftime('%Y-%m-%d') # Remove excess data
# chain = chain.rename_axis('Expirations') # Rename the index 

In [33]:
# sharpe ratio optimization
def optimal_sharpe(tickers, risk_free_rate, investment, data):
    # download data
    data = data 

    def neg_sharpe(weights):

        # determining number of shares of each stock that can be bought
        shares = []
        for i in range(len(tickers)):
            allocation = investment * weights[i]  # investment allocated to this stock
            price_per_share = data.iloc[0][tickers[i].upper()]
            
            # clculate fees
            flat_fee = 3.95
            per_share_fee = allocation/price_per_share/(1000+1/price_per_share) 
            
            # choose the smaller of the two fees
            trading_fee = min(flat_fee, per_share_fee)
            
            # calculate the number of shares after deducting the fee
            effective_investment = allocation - trading_fee
            shares.append(effective_investment / price_per_share)
        
        # forming the portfolio
        portfolio = data*shares
        portfolio['total'] = portfolio.sum(axis=1)
        portfolio['daily return'] = portfolio['total'].pct_change(1)

        # calculating sharpe ratio
        er = portfolio['daily return'].mean()
        std = portfolio['daily return'].std()
        sharpe_ratio = (er-risk_free_rate)/std
        
        sharpe_ratio = sharpe_ratio*(252**0.5) # annualizing sharpe ratio by trading days

        return -sharpe_ratio #make sharpe ratio negative for minimize function

    # constraints
    def check_sum(weights): 
        return np.sum(weights)-1 #returns 0 if weights sum up to 1
    constraints = {'type': 'eq', 'fun': check_sum}

    min_weight = 1/(2*len(tickers))
    max_weight = 0.4

    bounds = [(min_weight, max_weight)]*len(tickers)

    # initial guess
    init_guess = [1.0/len(tickers)]*len(tickers)

    results = minimize(neg_sharpe, init_guess, method="SLSQP", bounds=bounds, constraints=constraints)

    return results

In [36]:
# need to edit ticker_lst to get the actual stocks we will be using
optimal = optimal_sharpe(ticker_lst, 0, amount, data)
print(optimal.x)

[0.01351351 0.01351351 0.01351351 0.01351351 0.01351351 0.01351351
 0.01351351 0.01351351 0.01351351 0.01351351 0.01351351 0.01351351
 0.01351351 0.01351351 0.01351351 0.24152754 0.01351351 0.01351351
 0.28549947 0.01351351 0.01351351 0.01351351 0.01351351 0.01351351
 0.01351351 0.01351351 0.01351351 0.01351351 0.01351351 0.01351351
 0.01351351 0.01351351 0.01351351 0.01351351 0.01351351 0.01351351
 0.01351351]


In [17]:
# Code to output final dataframe to a CSV file called Stocks_Group_XX.csv
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']]
Stocks_Final.to_csv(f'Stocks_Group_{group}.csv', index=False)

In [18]:
Portfolio_Final

,Ticker,Price,Currency,Shares,Value,Weight


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

---
<p style="color: #004dd3">
Akram Jamil
</p>

<p style="color: #2C8CA9">
Jester Yang
</p>

<p style="color: #3cc19d;">
Annie Wong
</p>

---